In [1]:
import pandas as pd
import itertools


In [2]:
estaciones = pd.read_csv('../data/estaciones-de-bicicletas-publicas.csv',sep=';',usecols=[0,1,7])
print 'cantidad de estaciones:',estaciones.shape[0]
estaciones.head(2)

cantidad de estaciones: 196


,X,Y,NRO_EST
0,-58.392452,-34.583133,1
1,-58.374822,-34.592589,2


In [3]:
viajes = pd.read_csv('../data/recorridos-realizados-2017.csv',sep=';',usecols=[0,3,5])
viajes.head()

C:\Users\fgonzalez\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PERIODO,ORIGEN_ESTACION,DESTINO_ESTACION
0,201703,9,94
1,201705,166,83
2,201705,108,6
3,201705,154,17
4,201705,1,114


In [4]:
viajesAgosto = viajes.loc[viajes.PERIODO == 201708,['ORIGEN_ESTACION','DESTINO_ESTACION']].reset_index().drop(['index'],axis=1)
viajesAgosto['cantidad'] = 1
viajesAgosto.ORIGEN_ESTACION = viajesAgosto.ORIGEN_ESTACION.map(int)
print 'cantidad de estaciones unicas:', len(viajesAgosto.ORIGEN_ESTACION.unique())
viajesAgosto.head()

cantidad de estaciones unicas: 197


,ORIGEN_ESTACION,DESTINO_ESTACION,cantidad
0,18,146,1
1,77,76,1
2,20,125,1
3,112,146,1
4,52,85,1


In [5]:
print 'viajes que no estan en data set de estaciones:' 
viajesAgosto.ORIGEN_ESTACION.unique()[~(pd.Series(viajesAgosto.ORIGEN_ESTACION.unique()).isin(estaciones.NRO_EST.unique()))]

viajes que no estan en data set de estaciones:


array([102, 502, 529, 505, 513,  39, 200], dtype=int64)

In [6]:
#me quedo con los viajes cuyo origen Y destino estan en el dataset de Estaciones
mask = (viajesAgosto.ORIGEN_ESTACION.isin(estaciones.NRO_EST.unique())) & (viajesAgosto.DESTINO_ESTACION.isin(estaciones.NRO_EST.unique()))
viajesAgosto = viajesAgosto[mask]

In [7]:
print 'cantidad de estaciones unicas:', len(viajesAgosto.ORIGEN_ESTACION.unique())


cantidad de estaciones unicas: 190


In [8]:
viajesSuma = viajesAgosto.groupby(by=['ORIGEN_ESTACION','DESTINO_ESTACION']).sum().reset_index()
print 'fila viajeSuma:',viajesSuma.shape[0]
viajesSuma.head()

fila viajeSuma: 25209


,ORIGEN_ESTACION,DESTINO_ESTACION,cantidad
0,1,1,266
1,1,2,197
2,1,3,5
3,1,4,12
4,1,5,91


In [44]:
#genero la base de todos los pares i,j posibles para la matriz OD en base a las estaciones existentes
listaPosibles= list(itertools.product(estaciones.NRO_EST, estaciones.NRO_EST))
tablaMatriz =pd.DataFrame({'ORIGEN_ESTACION':[i[0] for i in listaPosibles],
                     'DESTINO_ESTACION':[i[1] for i in listaPosibles]})
print tablaMatriz.shape
tablaMatriz.head()

(38416, 2)


,DESTINO_ESTACION,ORIGEN_ESTACION
0,1,1
1,2,1
2,3,1
3,4,1
4,6,1


In [45]:
print estaciones.shape
tablaMatriz = tablaMatriz.merge(viajesSuma, left_on=['ORIGEN_ESTACION','DESTINO_ESTACION'], right_on=['ORIGEN_ESTACION','DESTINO_ESTACION'],how='left')
tablaMatriz.cantidad.fillna(0,inplace=True)
tablaMatriz.cantidad = tablaMatriz.cantidad.map(int)
print tablaMatriz.shape
tablaMatriz.head()

(196, 3)
(38416, 3)


,DESTINO_ESTACION,ORIGEN_ESTACION,cantidad
0,1,1,266
1,2,1,197
2,3,1,5
3,4,1,12
4,6,1,6


In [46]:
matriz = tablaMatriz.pivot(index='ORIGEN_ESTACION', columns='DESTINO_ESTACION', values='cantidad')
#matriz = matriz.fillna(0)

In [47]:
print matriz.shape
matriz.head()

(196, 196)


DESTINO_ESTACION,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,515
ORIGEN_ESTACION,,,,,,,,,,,,,,,,,,,,,
1,266,197,5,12,91,6,6,0,93,1,...,3,9,12,1,0,0,0,0,1,0
2,164,88,25,34,22,16,5,0,31,19,...,2,1,5,1,1,0,0,0,0,0
3,10,23,85,27,1,31,4,0,7,67,...,5,3,0,2,0,1,0,0,1,0
4,11,14,16,144,4,27,5,0,10,40,...,0,1,5,3,0,2,0,0,2,0
5,80,19,0,1,301,2,2,0,49,1,...,0,3,6,8,0,0,0,0,0,0


In [48]:
matriz.loc[:,[515]].sum()

DESTINO_ESTACION
515    0
dtype: int64

In [49]:
matriz.loc[[515],:].sum(axis=1)

ORIGEN_ESTACION
515    0
dtype: int64

In [50]:
matriz.to_csv('biciOD.csv')